In [3]:
import qiskit
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import numpy as np
from math import gcd
from fractions import Fraction

# --- 1. 设置经典参数 ---
N = 15
a = 7 # 我们选择 a=7, gcd(7, 15) = 1

# --- 2. 构建受控模乘电路 U^x |y> = |a^x * y mod N> ---
# 这是Shor算法中最难构建的部分。
# 对于 N=15, a=7, 我们可以手动构建这些门。
# 注意：这是一个为教学目的简化的版本，通用的模乘电路非常复杂。

def c_amod15(a, power):
    """受控的 U^power 操作"""
    U = QuantumCircuit(4)
    for _ in range(power):
        if a == 7:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        elif a == 11:
            U.swap(0, 2)
            U.swap(1, 3)
        # 其他a值的实现会更复杂
    U = U.to_gate()
    U.name = f"{a}^{power} mod {N}"
    c_U = U.control()
    return c_U

# --- 3. 构建找阶电路 ---
n_count = 8 # 计数比特数量，越多精度越高
n_target = 4  # 目标比特数量，足以表示15 (1111)

qc = QuantumCircuit(n_count + n_target, n_count)

# a. 初始化状态
# 将目标比特初始化为 |1>
qc.x(n_count)
# 对计数比特应用Hadamard门
for q in range(n_count):
    qc.h(q)

qc.barrier()

# b. 应用受控模乘门
for q in range(n_count):
    power = 2**q
    # 从最低位比特开始，依次应用 U, U^2, U^4, ...
    qc.append(c_amod15(a, power), [q] + list(range(n_count, n_count + n_target)))

qc.barrier()

# c. 应用逆量子傅里叶变换 (IQFT)
iqft = QFT(n_count, inverse=True)
qc.append(iqft, range(n_count))

qc.barrier()

# d. 测量
qc.measure(range(n_count), range(n_count))

# 我们可以画出电路图看看（电路会非常大）
# qc.draw(fold=-1)

# --- 4. 模拟并进行经典后处理 ---
backend = Aer.get_backend('qasm_simulator')
results = execute(qc, backend, shots=1024).result()
counts = results.get_counts()
# plot_histogram(counts) # 可以取消注释来查看测量结果的直方图

# --- 5. 从测量结果中推断阶 r ---
# 理想情况下，测量结果集中在 s/r 的近似值上
# s/r = 测量的十进制数 / 2^n_count
for measured_str in counts:
    # 将二进制测量结果转为十进制
    measured_int = int(measured_str, 2)
    phase = measured_int / (2**n_count)
    
    # 使用连分数算法找到最接近的分数 s/r
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    
    # 我们需要一个非0且偶数的r
    if r % 2 == 0 and r != 0:
        print(f"找到一个可能的阶 r = {r} (来自测量结果 {measured_str})")
        
        # --- 6. 计算因子 ---
        guess1 = gcd(a**(r//2) - 1, N)
        guess2 = gcd(a**(r//2) + 1, N)
        
        if guess1 != 1 and guess1 != N:
            print(f"成功！因子是: {guess1} 和 {guess2}")
            break # 找到一组有效因子就退出循环
else:
    print("在此次运行中未能从测量结果中找到有效的阶 r。")

ImportError: cannot import name 'Aer' from 'qiskit' (/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/qiskit/__init__.py)

In [3]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import Aer
from qiskit.compiler import transpile
from qiskit.providers.basic_provider import BasicProvider
from qiskit.circuit.library import QFT
from qiskit.circuit import Gate
import math

# ----------------------------------------------------------------------
# 模块 1: 底层构建块 - 受控加法器
# ----------------------------------------------------------------------

def qft_adder_gate(n_bits):
    """构造一个 n-bit 的量子傅里叶加法器。
    它将 |a>|b> 变换为 |a>|a+b>。
    """
    a = QuantumRegister(n_bits, name='a')
    b = QuantumRegister(n_bits, name='b')
    adder_qc = QuantumCircuit(a, b, name='QFT Adder')

    # 1. 对寄存器 b 进行 QFT
    adder_qc.append(QFT(n_bits, do_swaps=False), b)

    # 2. 应用受控相位旋转
    for i in range(n_bits):
        for j in range(i, n_bits):
            # 控制比特是 a[j-i]，目标比特是 b[j]
            # 相位是 2*pi*i / 2^(j-i+1)
            angle = 2 * np.pi / (2**(j - i + 1))
            adder_qc.cp(angle, a[j - i], b[j])

    # 3. 对寄存器 b 进行 IQFT
    adder_qc.append(QFT(n_bits, do_swaps=False, inverse=True), b)

    return adder_qc.to_gate()

def controlled_adder_gate(n_bits, controlled=True):
    """构造一个(受控的)加法门。"""
    ctrl = QuantumRegister(1, 'ctrl')
    a = QuantumRegister(n_bits, 'a')
    b = QuantumRegister(n_bits, 'b')
    
    if controlled:
        qc = QuantumCircuit(ctrl, a, b, name=f'C-ADD')
        adder_gate = qft_adder_gate(n_bits).control(1)
        qc.append(adder_gate, [ctrl[0]] + list(a) + list(b))
    else:
        qc = QuantumCircuit(a, b, name=f'ADD')
        adder_gate = qft_adder_gate(n_bits)
        qc.append(adder_gate, list(a) + list(b))
        
    return qc.to_gate()

# ----------------------------------------------------------------------
# 模块 2: 模算术 - 受控模加法器
# ----------------------------------------------------------------------

def controlled_modular_adder_gate(n_bits, N):
    """构造受控模加法器: |ctrl>|a>|b> -> |ctrl>|a>|(a+b) mod N>"""
    ctrl = QuantumRegister(1, 'ctrl')
    a = QuantumRegister(n_bits, 'a')
    b = QuantumRegister(n_bits, 'b')
    ancilla = QuantumRegister(1, 'ancilla') # 辅助比特用于比较
    N_reg = QuantumRegister(n_bits, 'N_reg') # 存储 N

    qc = QuantumCircuit(ctrl, a, b, ancilla, N_reg, name=f'C-ADD_mod{N}')

    # 1. (a+b)
    qc.append(controlled_adder_gate(n_bits), [ctrl[0]] + list(a) + list(b))

    # 2. -(a+b-N) = N-(a+b)
    # 首先将 N 加到 b 寄存器 (通过反向加法，即减法)
    # 我们需要一个 N 的比特串
    N_bits = f'{N:0{n_bits}b}'
    for i in range(n_bits):
        if N_bits[i] == '1':
            qc.x(N_reg[n_bits - 1 - i])
    
    # 减去 N (即加上 -N)
    add_inv_gate = qft_adder_gate(n_bits).inverse()
    qc.append(add_inv_gate, list(b) + list(N_reg))
    
    # 3. 检查 (a+b) >= N
    # 如果 b < 0 (即最高位为1)，说明 a+b < N，否则 a+b >= N
    qc.cnot(b[n_bits-1], ancilla[0])

    # 4. 如果 (a+b) >= N, 则 b 寄存器的值是 a+b-N, 我们需要加回 N
    # 这由 ancilla 控制
    qc.append(controlled_adder_gate(n_bits), [ancilla[0]] + list(N_reg) + list(b))
    
    # 5. 反计算，清理辅助比特
    # 恢复 b 寄存器
    qc.cnot(b[n_bits-1], ancilla[0])
    # 恢复 N 寄存器
    qc.append(add_inv_gate.inverse(), list(b) + list(N_reg))
    for i in range(n_bits):
        if N_bits[i] == '1':
            qc.x(N_reg[n_bits - 1 - i])

    # 6. 最后一步: 减去 a，将 b 恢复到初始状态
    # 这样整个电路的作用是 |a>|b> -> |a>|b+a mod N>
    add_a_inv_gate = qft_adder_gate(n_bits).inverse()
    qc.append(add_a_inv_gate.control(1), [ctrl[0]] + list(a) + list(b))

    return qc.to_gate()

# ----------------------------------------------------------------------
# 模块 3: 最终目标 - 受控模乘门
# ----------------------------------------------------------------------
def controlled_modular_multiplier_gate(a, n_bits, N):
    """
    构造受控模乘门: |ctrl>|x>|b> -> |ctrl>|x>|(x + b*a) mod N>
    这里 x 是累加器，b 是另一个乘数 (在Shor算法中通常是|0>)
    """
    ctrl = QuantumRegister(1, 'ctrl')
    x = QuantumRegister(n_bits, 'x') # 乘数寄存器
    b = QuantumRegister(n_bits, 'b') # 累加器
    ancilla = QuantumRegister(n_bits + 1, 'ancilla') # 辅助比特

    qc = QuantumCircuit(ctrl, x, b, ancilla, name=f'C-MULT({a})_mod{N}')

    # 通过一系列受控加法实现乘法
    for i in range(n_bits):
        # 计算 2^i * a mod N
        a_mod = (a * (2**i)) % N
        
        # 将 a_mod 编码到辅助寄存器
        a_mod_bits = f'{a_mod:0{n_bits}b}'
        for j in range(n_bits):
            if a_mod_bits[j] == '1':
                qc.x(ancilla[n_bits - 1 - j])

        # 受 x[i] 和主 ctrl 控制的模加法
        # 我们需要一个 Toffoli 门来组合两个控制
        qc.ccx(ctrl[0], x[i], ancilla[n_bits])
        
        # 使用 ancilla[n_bits] 作为控制
        # 注意：这里的 controlled_modular_adder_gate 需要修改为接受 n_bits+1 的寄存器
        # 为了简化，我们假设有这样一个门
        # C-ADD_mod(ancilla[0..n-1], b) controlled by ancilla[n]
        # qc.append(controlled_modular_adder_gate(...), [ancilla[n_bits]] + ...)

        # 反计算
        qc.ccx(ctrl[0], x[i], ancilla[n_bits])
        for j in range(n_bits):
            if a_mod_bits[j] == '1':
                qc.x(ancilla[n_bits - 1 - j])
        
        qc.barrier()

    return qc.to_gate()

# ----------------------------------------------------------------------
# 示例：构建一个简单的门并绘制
# ----------------------------------------------------------------------
n = 3 # 使用 3 个比特
N_val = 7
a_val = 4

# 1. 演示 QFT 加法器
print("绘制 QFT 加法器电路 (n=3)...")
qft_adder_circuit = QuantumCircuit(QuantumRegister(n, 'a'), QuantumRegister(n, 'b'))
qft_adder_circuit.append(qft_adder_gate(n), qargs=list(range(2*n)))
qft_adder_circuit.decompose().draw('mpl', style='iqx').show()

# 2. 演示受控模加法器 (这是一个非常复杂的电路)
# 注意：上面的实现是概念性的，一个完全可运行的分解会非常庞大。
# 它的核心思想是 加法 -> 减法 -> 检查 -> 条件加法 -> 反计算
print("\n受控模加法器的概念性电路构建完成。")
print("由于其极端复杂性，直接绘制分解图会导致图像无法阅读。")
print("我们将展示其高级结构。")

c_mod_add_gate = controlled_modular_adder_gate(n, N_val)
demo_qc = QuantumCircuit(QuantumRegister(1, 'ctrl'), 
                         QuantumRegister(n, 'a'), 
                         QuantumRegister(n, 'b'),
                         QuantumRegister(1, 'ancilla'),
                         QuantumRegister(n, 'N_reg'))
demo_qc.append(c_mod_add_gate, qargs=list(range(1 + n + n + 1 + n)))
demo_qc.draw('mpl', style='iqx', fold=-1).show()


# 3. 演示受控模乘法器的高级结构
print("\n受控模乘法器的高级结构构建完成。")
c_mod_mult_gate = controlled_modular_multiplier_gate(a_val, n, N_val)
mult_demo_qc = QuantumCircuit(QuantumRegister(1, 'ctrl'),
                              QuantumRegister(n, 'x'),
                              QuantumRegister(n, 'b'),
                              QuantumRegister(n+1, 'ancilla'))
mult_demo_qc.append(c_mod_mult_gate, qargs=list(range(1 + n + n + n + 1)))
mult_demo_qc.draw('mpl', style='iqx', fold=-1).show()



绘制 QFT 加法器电路 (n=3)...


/var/folders/7c/kwhn06r91h57xlqx9rqvvpmm0000gn/T/ipykernel_62621/4065203660.py:23: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  adder_qc.append(QFT(n_bits, do_swaps=False), b)
/var/folders/7c/kwhn06r91h57xlqx9rqvvpmm0000gn/T/ipykernel_62621/4065203660.py:34: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  adder_qc.append(QFT(n_bits, do_swaps=False, inverse=True), b)


QiskitError: 'One or more instructions cannot be converted to a gate. "QFT" is not a gate instruction'